## TF tunner samples

In [1]:
from processing import load_image_dataset

x_lbp_train, y_lbp_train = load_image_dataset('Datasets/espe/lbp_histograms - copia/train', (40, 40), True)
x_lbp_test, y_lbp_test = load_image_dataset('Datasets/espe/lbp_histograms - copia/test', (40, 40), True)
x_lbp_validation, y_lbp_validation = load_image_dataset('Datasets/espe/lbp_histograms - copia/validation', (40, 40), True)

x_masks_train, y_masks_train = load_image_dataset('Datasets/espe/masks/train', (40, 40), True)
x_masks_test, y_masks_test = load_image_dataset('Datasets/espe/masks/test', (40, 40), True)
x_masks_validation, y_masks_validation = load_image_dataset('Datasets/espe/masks/validation', (40, 40), True)

Using TensorFlow backend.


In [8]:
print("x train rgb shape: {} \t x train masks shape: {} \t same?: {}".format(x_lbp_train.shape, x_masks_train.shape, x_lbp_train.shape==x_masks_train.shape ))
print("x test rgb shape: {} \t x test masks shape: {}  \t same?: {}".format(x_lbp_test.shape, x_masks_test.shape, x_lbp_test.shape==x_masks_test.shape))
print("x validation rgb shape: {} \t x validation masks shape: {} \t same?: {}".format(x_lbp_validation.shape, x_masks_validation.shape, x_lbp_validation.shape==x_masks_validation.shape))

x train rgb shape: (10507, 40, 40, 1) 	 x train masks shape: (10507, 40, 40, 1) 	 same?: True
x test rgb shape: (2261, 40, 40, 1) 	 x test masks shape: (2261, 40, 40, 1)  	 same?: True
x validation rgb shape: (2527, 40, 40, 1) 	 x validation masks shape: (2527, 40, 40, 1) 	 same?: True


In [9]:
y_masks_train.shape[1]

7

In [2]:
import numpy as np
x_masks, y_masks = np.concatenate((x_masks_train/255, x_masks_validation/255), axis=0), np.concatenate((y_masks_train, y_masks_validation), axis=0)
# concatenation with normalization
x_lbp, y_lbp = np.concatenate((x_lbp_train/255, x_lbp_validation/255), axis=0), np.concatenate((y_lbp_train, y_lbp_validation), axis=0)

In [5]:
x_masks.shape, x_lbp.shape

((13034, 40, 40, 1), (13034, 40, 40, 1))

# First work models training

### Training LBP identification model

In [5]:
%%time
from utils.Models import *
from training.utils import cv_training

loss_function = tf.keras.losses.CategoricalCrossentropy()
model = None
model = individual_feature_model((40, 40, 1), y_masks_train.shape[1])

cv_training(model=model, n_splits=3, x_data=[x_lbp], y_data=y_lbp, path_to_save_results='models/texture-silhouette/own/textures', batch_size=300)

Train on 6951 samples, validate on 1738 samples
Epoch 1/20
6951/6951 [==============================] - 2s 245us/sample - loss: 1.9473 - acc: 0.1470 - val_loss: 1.9457 - val_acc: 0.1456
Epoch 2/20
6951/6951 [==============================] - 1s 126us/sample - loss: 1.9165 - acc: 0.2041 - val_loss: 1.8750 - val_acc: 0.2417
Epoch 3/20
6951/6951 [==============================] - 1s 126us/sample - loss: 1.7281 - acc: 0.3316 - val_loss: 1.7991 - val_acc: 0.2986
Epoch 4/20
6951/6951 [==============================] - 1s 127us/sample - loss: 1.4619 - acc: 0.4500 - val_loss: 1.4915 - val_acc: 0.4379
Epoch 5/20
6951/6951 [==============================] - 1s 127us/sample - loss: 1.1334 - acc: 0.5909 - val_loss: 1.4909 - val_acc: 0.4517
Epoch 6/20
6951/6951 [==============================] - 1s 127us/sample - loss: 0.9372 - acc: 0.6718 - val_loss: 1.3040 - val_acc: 0.5270
Epoch 7/20
6951/6951 [==============================] - 1s 125us/sample - loss: 0.7742 - acc: 0.7374 - val_loss: 1.5133 - va

<Figure size 432x288 with 0 Axes>

In [7]:
# evaluating model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
model = load_model('models/own/experiments/lbp_model/experiment_1/model_3.h5')
model.evaluate(x_lbp_test/255, y_lbp_test)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2261/2261 [==============================] - 1s 281us/sample - loss: 1.6914 - acc: 0.6249


[1.6914048448488386, 0.6249447]

### Training masks identification model

In [10]:
%%time
from utils.Models import *
from training.utils import cv_training

loss_function = tf.keras.losses.CategoricalCrossentropy()

input_shape = x_masks.shape[1:]
model = None
model = individual_feature_model(input_shape, y_masks_train.shape[1])

cv_training(model=model, n_splits=3, x_data=[x_masks], y_data=y_masks, path_to_save_results='models/texture-silhouette_2/silhouette')

Train on 6951 samples, validate on 1738 samples
Epoch 1/20
6951/6951 [==============================] - 1s 208us/sample - loss: 1.7968 - acc: 0.2695 - val_loss: 2.0247 - val_acc: 0.2532
Epoch 2/20
6951/6951 [==============================] - 1s 141us/sample - loss: 1.1146 - acc: 0.5975 - val_loss: 2.2591 - val_acc: 0.3015
Epoch 3/20
6951/6951 [==============================] - 1s 141us/sample - loss: 0.6887 - acc: 0.7603 - val_loss: 2.0618 - val_acc: 0.3596
Epoch 4/20
6951/6951 [==============================] - 1s 137us/sample - loss: 0.4725 - acc: 0.8350 - val_loss: 2.3608 - val_acc: 0.4010
Epoch 5/20
6951/6951 [==============================] - 1s 140us/sample - loss: 0.3454 - acc: 0.8812 - val_loss: 2.6488 - val_acc: 0.4246
Epoch 6/20
6951/6951 [==============================] - 1s 141us/sample - loss: 0.2492 - acc: 0.9128 - val_loss: 2.5566 - val_acc: 0.4143
Epoch 7/20
6951/6951 [==============================] - 1s 140us/sample - loss: 0.1757 - acc: 0.9417 - val_loss: 3.1121 - va

<Figure size 432x288 with 0 Axes>

In [ ]:
# evaluating model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
model = load_model('models/own/experiments/sillhouette_model/experiment_2/model_3.h5')
model.evaluate(x_masks_test, y_masks_test)

### Training double input images with siamese model

In [4]:
%%time
from utils.Models import *
from training.utils import cv_training

loss_function = tf.keras.losses.CategoricalCrossentropy()

model = None
model = prototype_model_for_reid((40, 40, 1), y_masks_train.shape[1])

cv_training(model=model, n_splits=3, x_data=[x_masks, x_lbp], y_data=y_lbp, path_to_save_results='models/texture-silhouette/own/combined', batch_size=60)

Train on 6951 samples, validate on 1738 samples
Epoch 1/20
6951/6951 [==============================] - 3s 472us/sample - loss: 1.9331 - acc: 0.1849 - val_loss: 1.9446 - val_acc: 0.1387
Epoch 2/20
6951/6951 [==============================] - 2s 318us/sample - loss: 1.8601 - acc: 0.3584 - val_loss: 1.9205 - val_acc: 0.2152
Epoch 3/20
6951/6951 [==============================] - 2s 325us/sample - loss: 1.7685 - acc: 0.4735 - val_loss: 1.8801 - val_acc: 0.2618
Epoch 4/20
6951/6951 [==============================] - 2s 318us/sample - loss: 1.6481 - acc: 0.6025 - val_loss: 1.8116 - val_acc: 0.3838
Epoch 5/20
6951/6951 [==============================] - 2s 317us/sample - loss: 1.5070 - acc: 0.7039 - val_loss: 1.7530 - val_acc: 0.4591
Epoch 6/20
6951/6951 [==============================] - 2s 319us/sample - loss: 1.3748 - acc: 0.7409 - val_loss: 1.7138 - val_acc: 0.4528
Epoch 7/20
6951/6951 [==============================] - 2s 321us/sample - loss: 1.2475 - acc: 0.7730 - val_loss: 1.6124 - va

<Figure size 432x288 with 0 Axes>

In [5]:
# evaluating model
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
model = load_model('models/texture-silhouette/own/combined/model_3.h5')
model.evaluate([x_masks_test/255, x_lbp_test/255], y_masks_test)

2261/2261 [==============================] - 1s 246us/sample - loss: 1.8518 - acc: 0.4299


[1.8518066626636922, 0.42989826]

# Market-1501

In [2]:
from processing import load_image_dataset

x_lbp_train, y_lbp_train = load_image_dataset('Datasets/Market-1501/Market-1501-lbp/train_all', (40, 40), True)

x_masks_train, y_masks_train = load_image_dataset('Datasets/Market-1501/Market-1501-masks/train_all', (40, 40), True)

In [2]:
print("x train rgb shape: {} \t x train masks shape: {} \t same?: {}".format(x_lbp_train.shape, x_masks_train.shape, x_lbp_train.shape==x_masks_train.shape ))

x train rgb shape: (10784, 40, 40, 1) 	 x train masks shape: (10784, 40, 40, 1) 	 same?: True


In [3]:
len(y_masks_train[0]), y_masks_train.shape

(751, (10784, 751))

## Training silhouette re-id model

In [4]:
%%time
from utils.Models import individual_feature_model
from training.utils import cv_training

model = individual_feature_model((40, 40, 1), y_masks_train.shape[1])
cv_training(model=model, n_splits=3, x_data=[x_masks_train/255]   , y_data=y_lbp_train, path_to_save_results='models/texture-silhouette/market-1501/silhouette')

Train on 5751 samples, validate on 1438 samples
Epoch 1/20
5751/5751 [==============================] - 1s 213us/sample - loss: 6.4166 - acc: 0.0064 - val_loss: 7.5696 - val_acc: 0.0000e+00
Epoch 2/20
5751/5751 [==============================] - 1s 142us/sample - loss: 6.2293 - acc: 0.0054 - val_loss: 11.8494 - val_acc: 0.0000e+00
Epoch 3/20
5751/5751 [==============================] - 1s 144us/sample - loss: 6.1919 - acc: 0.0077 - val_loss: 9.3562 - val_acc: 0.0000e+00
Epoch 4/20
5751/5751 [==============================] - 1s 143us/sample - loss: 6.1211 - acc: 0.0117 - val_loss: 13.5038 - val_acc: 0.0000e+00
Epoch 5/20
5751/5751 [==============================] - 1s 152us/sample - loss: 5.9993 - acc: 0.0144 - val_loss: 11.9416 - val_acc: 0.0000e+00
Epoch 6/20
5751/5751 [==============================] - 1s 151us/sample - loss: 5.8347 - acc: 0.0202 - val_loss: 14.4831 - val_acc: 0.0000e+00
Epoch 7/20
5751/5751 [==============================] - 1s 148us/sample - loss: 5.6074 - acc: 0.

<Figure size 432x288 with 0 Axes>

## Training textures re-id model

In [7]:
%%time
from utils.Models import individual_feature_model
from training.utils import cv_training
model = None
model = individual_feature_model((40, 40, 1), y_masks_train.shape[1])
cv_training(model=model, n_splits=3, x_data=[x_lbp_train/255], y_data=y_lbp_train, path_to_save_results='models/texture-silhouette/market-1501/texture')

Train on 5751 samples, validate on 1438 samples
Epoch 1/20
5751/5751 [==============================] - 1s 241us/sample - loss: 6.4114 - acc: 0.0066 - val_loss: 7.9651 - val_acc: 0.0000e+00
Epoch 2/20
5751/5751 [==============================] - 1s 144us/sample - loss: 6.2243 - acc: 0.0077 - val_loss: 8.6683 - val_acc: 0.0000e+00
Epoch 3/20
5751/5751 [==============================] - 1s 145us/sample - loss: 6.2005 - acc: 0.0047 - val_loss: 9.7715 - val_acc: 0.0000e+00
Epoch 4/20
5751/5751 [==============================] - 1s 143us/sample - loss: 6.1921 - acc: 0.0083 - val_loss: 10.6870 - val_acc: 0.0000e+00
Epoch 5/20
5751/5751 [==============================] - 1s 147us/sample - loss: 6.1792 - acc: 0.0078 - val_loss: 16.2131 - val_acc: 0.0000e+00
Epoch 6/20
5751/5751 [==============================] - 1s 144us/sample - loss: 6.1760 - acc: 0.0085 - val_loss: 11.3269 - val_acc: 0.0000e+00
Epoch 7/20
5751/5751 [==============================] - 1s 143us/sample - loss: 6.1651 - acc: 0.0

<Figure size 432x288 with 0 Axes>

## Double features re-id model

In [14]:
%%time
from utils.Models import prototype_model_for_reid
from training.utils import cv_training
model=None
model = prototype_model_for_reid((40, 40, 1), y_masks_train.shape[1])
cv_training(model=model, n_splits=3, x_data=[x_masks_train/255, x_lbp_train/255]   , y_data=y_lbp_train, path_to_save_results='models/texture-silhouette/market-1501/combined', batch_size=20, epochs=20)

Train on 5751 samples, validate on 1438 samples
Epoch 1/20
5751/5751 [==============================] - 5s 837us/sample - loss: 6.5573 - acc: 0.0037 - val_loss: 6.9236 - val_acc: 0.0000e+00
Epoch 2/20
5751/5751 [==============================] - 4s 610us/sample - loss: 6.4257 - acc: 0.0061 - val_loss: 7.2695 - val_acc: 0.0000e+00
Epoch 3/20
5751/5751 [==============================] - 4s 620us/sample - loss: 6.3331 - acc: 0.0064 - val_loss: 7.6356 - val_acc: 0.0000e+00
Epoch 4/20
5751/5751 [==============================] - 4s 614us/sample - loss: 6.2688 - acc: 0.0082 - val_loss: 8.0204 - val_acc: 0.0000e+00
Epoch 5/20
5751/5751 [==============================] - 4s 628us/sample - loss: 6.2278 - acc: 0.0082 - val_loss: 8.4160 - val_acc: 0.0000e+00
Epoch 6/20
5751/5751 [==============================] - 4s 629us/sample - loss: 6.2022 - acc: 0.0082 - val_loss: 8.8198 - val_acc: 0.0000e+00
Epoch 7/20
5751/5751 [==============================] - 4s 626us/sample - loss: 6.1871 - acc: 0.0082

<Figure size 432x288 with 0 Axes>